In [14]:
import torch
masked_state_dict = torch.load('masked_gpt2.pt')
N=100

In [15]:
from transformer_lens.HookedTransformer import HookedTransformer
gpt2_masked = HookedTransformer.from_pretrained(is_masked=True, model_name='gpt2')

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [16]:
from transformer_lens.ioi_dataset import IOIDataset

ioi_dataset = IOIDataset(prompt_type="ABBA", N=N, nb_templates=1)
train_data = ioi_dataset.toks.long()


In [17]:
def logit_diff_from_ioi_dataset(
    logits: torch.Tensor, tokens: torch.Tensor, mean=False,
):
    assert tokens.shape == (
        N,
        16,
    ), tokens.shape  # TODO check this is not breaking things...
    assert len(logits.shape) == 3, logits.shape

    io_labels = tokens[:, 2]
    s_labels = tokens[:, 4]

    io_logits = logits[torch.arange(N), -2, io_labels]
    s_logits = logits[torch.arange(N), -2, s_labels]

    logit_diff = io_logits - s_logits
    if mean:
        return logit_diff.mean()
    else:
        return logit_diff


In [27]:
gpt2_masked.load_state_dict(masked_state_dict)
logit_diff_from_ioi_dataset(gpt2_masked(train_data), train_data, mean=True)

tensor(20.8944, device='cuda:0', grad_fn=<MeanBackward0>)